In [16]:
import pandas as pd
from streaming_optimizer import optimize_streaming_packages
from datetime import datetime

In [ ]:
games_raw = pd.read_csv('data/bc_game.csv') # some games and when they are played
streaming_packages_raw = pd.read_csv('data/bc_streaming_package.csv') # streaming packages with price
streaming_offers_raw = pd.read_csv('data/bc_streaming_offer.csv')  # which service offers which game, live? on-demand-highlights?
games_raw

,id,team_home,team_away,starts_at,tournament_name
0,1,Deutschland,Schottland,2024-06-14 19:00:00,Europameisterschaft 2024
1,2,Ungarn,Schweiz,2024-06-15 13:00:00,Europameisterschaft 2024
2,3,Spanien,Kroatien,2024-06-15 16:00:00,Europameisterschaft 2024
3,4,Italien,Albanien,2024-06-15 19:00:00,Europameisterschaft 2024
4,5,Polen,Niederlande,2024-06-16 13:00:00,Europameisterschaft 2024
...,...,...,...,...,...
8866,8872,San Tirso SD,Selaya FC,2024-10-09 19:00:00,Copa del Rey 24/25
8867,8873,UD San Pedro,CD Ceuta 6 de Junio,2024-10-09 19:00:00,Copa del Rey 24/25
8868,8874,CD Sonseca,Cp Parla Escuela,2024-10-09 19:00:00,Copa del Rey 24/25
8869,8875,CD Gevora,UD Playas de Sotavento,2024-10-09 19:00:00,Copa del Rey 24/25


In [15]:
game_ids_of_interest = list(range(31, 39)) # Input from the FE

,id,name,monthly_price_cents,monthly_price_yearly_subscription_in_cents
1,3,ZDF - Free-TV,0.0,0
2,4,ARD - Free-TV,0.0,0
5,13,Joyn - Standard,0.0,0
8,16,Zattoo - FREE,0.0,0
17,39,DAZN - Free,0.0,0
21,43,Sport1 - Free-TV,0.0,0
22,44,Sat1 - Free-TV,0.0,0
23,45,Sportdigital - Free,0.0,0
26,48,RTL - Free-TV,0.0,0


In [11]:
games_raw[games_raw['team_home'] == 'Bayern München']

[{'package': 3, 'start_date': datetime.date(2024, 9, 25)},
 {'package': 3, 'start_date': datetime.date(2024, 10, 1)},
 {'package': 3, 'start_date': datetime.date(2024, 10, 20)},
 {'package': 3, 'start_date': datetime.date(2024, 10, 23)},
 {'package': 3, 'start_date': datetime.date(2024, 11, 3)},
 {'package': 3, 'start_date': datetime.date(2024, 11, 26)},
 {'package': 3, 'start_date': datetime.date(2025, 1, 26)},
 {'package': 3, 'start_date': datetime.date(2025, 1, 29)},
 {'package': 13, 'start_date': datetime.date(2024, 9, 25)},
 {'package': 13, 'start_date': datetime.date(2024, 10, 1)},
 {'package': 13, 'start_date': datetime.date(2024, 10, 20)},
 {'package': 13, 'start_date': datetime.date(2024, 10, 23)},
 {'package': 13, 'start_date': datetime.date(2024, 11, 3)},
 {'package': 13, 'start_date': datetime.date(2024, 11, 26)},
 {'package': 13, 'start_date': datetime.date(2025, 1, 26)},
 {'package': 13, 'start_date': datetime.date(2025, 1, 29)},
 {'package': 16, 'start_date': datetime.da

In [37]:
## Preprocess data (make everthing as small as possible)


### Filter packages to include only relevant ones
# Identify relevant package IDs from offers
relevant_package_ids = streaming_offers_raw[streaming_offers_raw['game_id'].isin(game_ids_of_interest)]['streaming_package_id'].unique()

# Filter packages to include only relevant ones
filtered_packages = streaming_packages_raw[streaming_packages_raw['id'].isin(relevant_package_ids)]

games = games_raw[games_raw['id'].isin(game_ids_of_interest)]

# Filter offers to include only the games of interest
filtered_offers = streaming_offers_raw[streaming_offers_raw['game_id'].isin(game_ids_of_interest)]


# Extract unique package IDs
packages = filtered_packages['id'].unique().tolist()

# Create game_dates dictionary
games.loc[:, 'starts_at'] = pd.to_datetime(games['starts_at'])
game_dates = games.set_index('id')['starts_at'].apply(lambda x: x.date()).to_dict()


# Create C_month and C_year dictionaries, dropping packages with NA for the respective type
C_month = filtered_packages.dropna(subset=['monthly_price_cents']) \
    .set_index('id')['monthly_price_cents'].to_dict()
C_year = filtered_packages.dropna(subset=['monthly_price_yearly_subscription_in_cents']) \
    .set_index('id')['monthly_price_yearly_subscription_in_cents'].to_dict()

# Create P_g dictionary
P_g = filtered_offers.groupby('game_id')['streaming_package_id'].apply(list).to_dict()

results = optimize_streaming_packages(packages, game_ids_of_interest, game_dates, C_month, C_year, P_g)

if results is not None:
    # Print results
    print("Status:", results["status"])
    print("Total Cost:", results["total_cost"])
    print("Active Monthly Subscriptions:")
    for sub in results["active_monthly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
    print("Active Yearly Subscriptions:")
    for sub in results["active_yearly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
else:
    print("No results returned from optimize_streaming_packages function.")

results['active_monthly_subscriptions']
unique_package_ids = list({sub['package'] for sub in results['active_monthly_subscriptions']})
unique_package_ids

unique_packages = filtered_packages[filtered_packages['id'].isin(unique_package_ids)]
filtered_packages

Status: Optimal
Total Cost: 1999.0
Active Monthly Subscriptions:
  Package: 3, Start Date: 2024-09-25
  Package: 3, Start Date: 2024-10-01
  Package: 3, Start Date: 2024-10-20
  Package: 3, Start Date: 2024-10-23
  Package: 3, Start Date: 2024-11-03
  Package: 3, Start Date: 2024-11-26
  Package: 3, Start Date: 2025-01-26
  Package: 3, Start Date: 2025-01-29
  Package: 13, Start Date: 2024-09-25
  Package: 13, Start Date: 2024-10-01
  Package: 13, Start Date: 2024-10-20
  Package: 13, Start Date: 2024-10-23
  Package: 13, Start Date: 2024-11-03
  Package: 13, Start Date: 2024-11-26
  Package: 13, Start Date: 2025-01-26
  Package: 13, Start Date: 2025-01-29
  Package: 16, Start Date: 2024-09-25
  Package: 16, Start Date: 2024-10-01
  Package: 16, Start Date: 2024-10-20
  Package: 16, Start Date: 2024-10-23
  Package: 16, Start Date: 2024-11-03
  Package: 16, Start Date: 2024-11-26
  Package: 16, Start Date: 2025-01-26
  Package: 16, Start Date: 2025-01-29
  Package: 39, Start Date: 2024

,id,name,monthly_price_cents,monthly_price_yearly_subscription_in_cents
0,2,MagentaTV - MegaSport,NaN,6000
1,3,ZDF - Free-TV,0.0,0
5,13,Joyn - Standard,0.0,0
6,14,waipu.tv - Comfort,749.0,749
7,15,Vodafone - GigaTV Cable,NaN,1249
8,16,Zattoo - FREE,0.0,0
9,17,Amazon - Prime Video,899.0,749
10,18,DAZN - Unlimited,4499.0,3499
12,20,Zattoo - SMART HD,649.0,649
15,37,DAZN - World,999.0,699


In [26]:
from pandas import Timestamp

packages = [2, 3, 4, 10, 13, 14, 15, 16, 17, 18, 19]
games = [1, 2, 3] #game_ids_of_interest
game_dates = {1: datetime(2024, 6, 14),
              2: datetime(2024, 6, 15),
              3: datetime(2024, 6, 15)}
C_month = {3: 0.0,
 4: 0.0,
 10: 3599.0,
 13: 0.0,
 14: 749.0,
 16: 0.0,
 17: 899.0,
 18: 4499.0}
C_year = {2: 6000,
 3: 0,
 4: 0,
 10: 2999,
 13: 0,
 14: 749,
 15: 1249,
 16: 0,
 17: 749,
 18: 3499,
 19: 3000}

P_g = {1: [2, 3, 4, 10, 13, 14, 15, 16, 17, 18, 19],
 2: [2, 3, 4, 10],
 3: [2, 3, 4, 10, 13, 14, 15, 16, 17, 18, 19]}

# optimize_streaming_packages(packages, games, game_dates, C_month, C_year, P_g)
game_dates

{1: datetime.datetime(2024, 6, 14, 0, 0),
 2: datetime.datetime(2024, 6, 15, 0, 0),
 3: datetime.datetime(2024, 6, 15, 0, 0)}

In [17]:
from streaming_optimizer import optimize_streaming_packages
from datetime import datetime, timedelta

# Example Input Data
packages = ["P1", "P2"]
games = ["G1", "G2", "G3", "G4", "G5", 'G6', 'G7']
game_dates = {
    "G1": datetime(2023, 1, 15),
    "G2": datetime(2023, 2, 20),
    "G3": datetime(2023, 3, 25),
    "G4": datetime(2023, 4, 10),
    "G5": datetime(2023, 4, 20),
    "G6": datetime(2024, 1, 10),
    "G7": datetime(2024, 3, 15),
}
C_month = {'P1': 20}
C_year = {'P1': 180}
P_g = {'G1': ['P1'], 'G2': ['P1'], 'G3': ['P1', 'P2'], 'G4': ['P1', 'P2'], 'G5': ['P1', 'P2'],
    'G6': ['P1'], 'G7': ['P1', 'P2']}
results = optimize_streaming_packages(packages, games, game_dates, C_month, C_year, P_g)


if results is not None:
    # Print results
    print("Status:", results["status"])
    print("Total Cost:", results["total_cost"])
    print("Active Monthly Subscriptions:")
    for sub in results["active_monthly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
    print("Active Yearly Subscriptions:")
    for sub in results["active_yearly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
else:
    print("No results returned from optimize_streaming_packages function.")



Status: Optimal
Total Cost: 100.0
Active Monthly Subscriptions:
  Package: P1, Start Date: 2023-01-15
  Package: P1, Start Date: 2023-02-20
  Package: P1, Start Date: 2023-03-25
  Package: P1, Start Date: 2024-01-10
  Package: P1, Start Date: 2024-03-15
Active Yearly Subscriptions:
